In [1]:
db_name = 'ted-talks/ted-talks.db'

# OCHO Properties

In [2]:
OHCO = ['speaker', 'event', 'id', 'para_num', 'sent_num', 'token_num']
AUTHS = OHCO[:1]
BOOKS = OHCO[:2]
CHAPS = OHCO[:3]
PARAS = OHCO[:4]
SENTS = OHCO[:5]

BAG = CHAPS

In [3]:
import sqlite3
import pandas as pd
import numpy as np

In [4]:
with sqlite3.connect(db_name) as db:
    tokens = pd.read_sql('SELECT * FROM token', db, index_col=OHCO)
    vocab = pd.read_sql('SELECT * FROM vocab', db, index_col='term_id')
    docs =  pd.read_sql('SELECT * FROM doc', db, index_col=CHAPS)

In [5]:
WORDS = (tokens.punc == 0) & (tokens.num == 0)
BOW = tokens[WORDS].groupby(BAG + ['term_id'])['term_id'].count()

In [6]:
DTM = BOW.unstack().fillna(0)
bags = pd.DataFrame(index = DTM.index)
DTM = DTM.reset_index(drop=True)
DTM.index.name = 'bag_id'

In [7]:
N_docs = DTM.shape[0]
vocab['df'] = DTM[DTM > 0].count()
vocab['idf'] = np.log10(N_docs / vocab.df)
# vocab[vocab.stop==0].sort_values('n', ascending=False).head(500).sort_values('idf', ascending=False).head(20)

In [8]:
TFIDF = DTM * vocab['idf']
vocab[TFIDF.sum() == 0]
vocab['tfidf_sum'] = TFIDF.sum()
vocab['tfidf_mean'] = TFIDF.mean()
vocab['tfidf_max'] = TFIDF.max()

In [9]:
def get_top_terms(vocab, no_stops=True, sort_col='n', k=1000):
    if no_stops:
        V = vocab[vocab.stop == 0]
    else:
        V = vocab
    return V.sort_values(sort_col, ascending=False).head(k)

In [59]:
# proper_nouns = tokens.loc[tokens.pos == 'NNP', 'term_id'].unique()
top_n = 100
TOPV = get_top_terms(vocab, sort_col='n', k=top_n)
# TOPV = get_top_terms(vocab.loc[vocab.index.isin(proper_nouns)], sort_col='n', k=top_n)

In [62]:
tfidf_small = TFIDF[TOPV.index].stack().to_frame().rename(columns={0:'w'})

bag_id  term_id
2436    73937       0.000000
968     66354       0.000000
        65135       0.000000
        27102       0.000000
        37450       0.000000
        57410       0.000000
2198    65797       0.000000
968     37061       0.000000
        43668       0.000000
        4597        0.000000
2198    30688       0.000000
968     37804       0.000000
2198    17224       0.000000
        23449       0.000000
968     44012       0.000000
2198    33784       0.000000
968     56909       0.000000
        68714       0.000000
2198    62691       0.000000
        6084        0.000000
        31272       0.000000
968     55799       0.000000
        26820       0.000000
        71903       0.000000
        38746       0.000000
        66387       0.000000
        72408       0.000000
        581         0.000000
        73955       0.000000
2198    6252        0.000000
                     ...    
1297    7795       21.369898
261     7795       21.369898
1295    7795       22.08222

In [17]:
tokens.head()

In [13]:
vocab.head()

,term_str,n,p,port_stem,stop,df,idf,tfidf_sum,tfidf_mean,tfidf_max
term_id,,,,,,,,,,
0,a,149535,2.416183e-02,a,1,3361,0.000775,115.830397,0.034402,0.166540
1,aa,10,1.615797e-06,aa,0,5,2.828273,28.282731,0.008400,8.484819
2,aaa,9,1.454218e-06,aaa,0,5,2.828273,25.454458,0.007560,14.141366
3,aaaaa,2,3.231595e-07,aaaaa,0,2,3.226213,6.452426,0.001916,3.226213
4,aaaaaaaah,1,1.615797e-07,aaaaaaaah,0,1,3.527243,3.527243,0.001048,3.527243


In [14]:
docs.head()

,,,transcript
speaker,event,id,
Crush Club,TED Salon: Radical Craft,35945,\nTC Milan: How you doing tonight? New York!\n...
JR,Torchbearers,13010,JR: Art is not supposed to change the world. B...
OK Go,TED2017,2792,\n\n\n\n\nYou know you can't keep letting it g...
Sauti Sol,TEDGlobal 2017,12803,\n Ooh ooh\nShe wants to be rich\nShe wants to...
A Tribe Called Red,TED2018,26265,\nWe are the tribe that they cannot see\nWe li...


In [16]:
bags.head()

,,
speaker,event,id
Crush Club,TED Salon: Radical Craft,35945
JR,Torchbearers,13010
OK Go,TED2017,2792
Sauti Sol,TEDGlobal 2017,12803
A Tribe Called Red,TED2018,26265


In [ ]:
with sqlite3.connect(db_name) as db:
    vocab.to_sql('vocab', db, if_exists='replace', index=True)
    tokens.to_sql('token', db, if_exists='replace', index=True)
    docs.to_sql('doc', db, if_exists='replace', index=True)
    tfidf_small.to_sql('tfidf_small', db, if_exists='replace', index=True)
    bags.reset_index().to_sql('bag', db, if_exists='replace', index=True, index_label='bag_id')